In [1]:
# import the require libraries 
from bs4 import BeautifulSoup
from urllib import request
from google.colab import files
import re
import numpy as np
import time
import os

domain_name = 'https://www.gsmarena.com/'

In [2]:
debug_collector = []

def collect_debug(error):
  '''A function for logging any unexpected behaviour'''
  global debug_collector
#   print(error)
  debug_collector.append(error)

In [3]:
# STEP 1
def get_maker_links(url):
  page = request.urlopen(url)
  html = page.read()
  

  bs = BeautifulSoup(html, 'html.parser')


  table = bs.findChild('div', class_='st-text').table


  if not table:
    error = 'Maker Page Error: has no brands table| function_name: {}| url: {}'.format(function_name, url)
    collect_debug(error)

  else:

    rows = table.findChildren('td')
    rows_collector = []

    for maker_id, row in enumerate(rows):

      row_a_tag = row.a
      if not row_a_tag:
        error = 'Maker Page Error: no row_a_tag| function_name: {}| url: {}| row_num: {}'.format(function_name, url, n)
        collect_debug(error)
      else:
        maker_link = domain_name + row_a_tag['href']
        maker_name, num_devices = (item for item in row_a_tag.stripped_strings)


        num_devices = re.findall(re.compile('\d+'), num_devices)[0]
        num_devices = int(num_devices)

        rows_collector.append([maker_id, maker_name, num_devices, maker_link])
  return rows_collector

In [4]:
seed_path = 'makers.php3'
seed_url = domain_name + seed_path 


maker_list = get_maker_links(seed_url)
print(maker_list)

[[0, 'Acer', 100, 'https://www.gsmarena.com/acer-phones-59.php'], [1, 'alcatel', 407, 'https://www.gsmarena.com/alcatel-phones-5.php'], [2, 'Allview', 157, 'https://www.gsmarena.com/allview-phones-88.php'], [3, 'Amazon', 22, 'https://www.gsmarena.com/amazon-phones-76.php'], [4, 'Amoi', 47, 'https://www.gsmarena.com/amoi-phones-28.php'], [5, 'Apple', 111, 'https://www.gsmarena.com/apple-phones-48.php'], [6, 'Archos', 43, 'https://www.gsmarena.com/archos-phones-90.php'], [7, 'Asus', 196, 'https://www.gsmarena.com/asus-phones-46.php'], [8, 'AT&T', 4, 'https://www.gsmarena.com/at&t-phones-57.php'], [9, 'Benefon', 9, 'https://www.gsmarena.com/benefon-phones-15.php'], [10, 'BenQ', 35, 'https://www.gsmarena.com/benq-phones-31.php'], [11, 'BenQ-Siemens', 27, 'https://www.gsmarena.com/benq_siemens-phones-42.php'], [12, 'Bird', 61, 'https://www.gsmarena.com/bird-phones-34.php'], [13, 'BlackBerry', 92, 'https://www.gsmarena.com/blackberry-phones-36.php'], [14, 'Blackview', 47, 'https://www.gsmare

In [5]:

def get_makers_link(maker_name):
  global maker_list
  if not maker_list:
      error = 'No maker_list!'
      collect_debug(error)
  else:
    for list_item in maker_list:
      if maker_name.lower() == list_item[1].lower():
        print('maker_link called for {} \n{}'.format(maker_name, list_item[-1]))
        
  
# checking
maker_name = 'Samsung'
get_makers_link(maker_name)

maker_name = 'samSung'
get_makers_link(maker_name)

maker_link called for Samsung 
https://www.gsmarena.com/samsung-phones-9.php
maker_link called for samSung 
https://www.gsmarena.com/samsung-phones-9.php


In [6]:

def get_bs(url, parser='html.parser'):

  page = request.urlopen(url)
  html = page.read()
  bs = BeautifulSoup(html, parser)
  return bs

### Expanding The Capability Of The Web Crawler

In [7]:

def get_largest_maker():
  global maker_list
  is_largest = 0
  maker_id = None
  for maker in maker_list:
    num_devices = maker[2]
    if num_devices > is_largest:
      is_largest = num_devices
      maker_id = maker[0]
  return maker_list[maker_id]

# checking
maker = get_largest_maker()
print(maker)

[87, 'Samsung', 1343, 'https://www.gsmarena.com/samsung-phones-9.php']


In [8]:
# STEP 2

def get_nav_page_links(maker):
  maker_id, maker_name, num_devices, maker_link = maker
  maker_nav_pages = {}

  maker_nav_pages[1] = maker_link

  bs = get_bs(maker_link)


  nav_pages = bs.findChild('div', class_='nav-pages')

  if not nav_pages:
    error = '{} does not have nav_pages| maker_link: {}'.format(maker_name, maker_link)
    collect_debug(error)

  else:

    nav_pages = nav_pages.findChildren('a', recursive=False)
    for nav_page_num, nav_page in enumerate(nav_pages):

        maker_nav_pages[nav_page_num + 2] = domain_name + nav_page['href']
  return maker_nav_pages

# checking
nav_page_links = get_nav_page_links(maker)  
print(nav_page_links)  

{1: 'https://www.gsmarena.com/samsung-phones-9.php', 2: 'https://www.gsmarena.com/samsung-phones-f-9-0-p2.php', 3: 'https://www.gsmarena.com/samsung-phones-f-9-0-p3.php', 4: 'https://www.gsmarena.com/samsung-phones-f-9-0-p4.php', 5: 'https://www.gsmarena.com/samsung-phones-f-9-0-p5.php', 6: 'https://www.gsmarena.com/samsung-phones-f-9-0-p6.php', 7: 'https://www.gsmarena.com/samsung-phones-f-9-0-p7.php', 8: 'https://www.gsmarena.com/samsung-phones-f-9-0-p8.php', 9: 'https://www.gsmarena.com/samsung-phones-f-9-0-p9.php', 10: 'https://www.gsmarena.com/samsung-phones-f-9-0-p10.php', 11: 'https://www.gsmarena.com/samsung-phones-f-9-0-p11.php', 12: 'https://www.gsmarena.com/samsung-phones-f-9-0-p12.php', 13: 'https://www.gsmarena.com/samsung-phones-f-9-0-p13.php', 14: 'https://www.gsmarena.com/samsung-phones-f-9-0-p14.php', 15: 'https://www.gsmarena.com/samsung-phones-f-9-0-p15.php', 16: 'https://www.gsmarena.com/samsung-phones-f-9-0-p16.php'}


Once the **nav_pages** for a maker (Samsung in this case) have been collected, we can collect the device info and most importantly the **device_link** for every device by that maker.

In [9]:
# STEP 3
devices_collector = {}
maker_name = maker[1]
maker_link = maker[-1]


nav_page_links = get_nav_page_links(maker) 


devices_collector[maker_name] = [] 
def get_device_links(nav_page_link, devices_collector, maker_name):
  bs = get_bs(nav_page_link)

  devices = bs.findChild('div', class_='makers').ul
  devices = devices.findChildren('li', recrusive=False)
  page_device_collector = []
  for device_num, device in enumerate(devices):
    device_name = device.get_text()
    if not device.a:
        error = "{} does not have a link : nav_page: {}| maker_name {}: ".format(device_name, nav_page_link , maker_name)
        collect_debug.append(error)
    else:
      device_link = domain_name + device.a['href']
      img_tag = device.a.findChild('img')
      if not img_tag:
        error = "{} does not have a img_tag| nav_page: {}| maker_name: {}".format(device_name, nav_page_link , maker_name)
        collect_debug.append(error)
      else:
        device_img_link = img_tag['src']
        device_info = img_tag['title']
    page_device_collector.append([device_name, device_info, device_img_link, device_link]) 

  devices_collector[maker_name] += page_device_collector

# checking
for nav_page_num, nav_page_link in nav_page_links.items():
  get_device_links(nav_page_link, devices_collector, maker_name)
  
print(devices_collector.keys())
print(devices_collector['Samsung'])
print(devices_collector['Samsung'].__len__())

dict_keys(['Samsung'])
[['Galaxy A04e', 'Samsung Galaxy A04e Android smartphone. Announced Oct 2022. Features 6.5″  display, 5000 mAh battery, 128 GB storage, 4 GB RAM.', 'https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-a04e.jpg', 'https://www.gsmarena.com/samsung_galaxy_a04e-11945.php'], ['Galaxy Tab Active4 Pro', 'Samsung Galaxy Tab Active4 Pro Android tablet. Announced Aug 2022. Features 10.1″  display, Snapdragon 778G 5G chipset, 7600 mAh battery, 128 GB storage, 4 GB RAM, MIL-STD-810H compliant, Corning Gorilla Glass 5.', 'https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-tab-active4-pro.jpg', 'https://www.gsmarena.com/samsung_galaxy_tab_active4_pro-11840.php'], ['Galaxy A04s', 'Samsung Galaxy A04s Android smartphone. Announced Aug 2022. Features 6.5″  display, Exynos 850 chipset, 5000 mAh battery, 128 GB storage, 4 GB RAM.', 'https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-a04s.jpg', 'https://www.gsmarena.com/samsung_galaxy_a04s-11803.php'], ['Galaxy A04', 'Samsung Galaxy 

In [10]:
devices = devices_collector['Samsung']
device = devices[3]


device_link = device[-1]
device_name = device[0]


specs_collector = {}

In [11]:

def get_device_specs(bs, specs_collector, device_name, device_link):
  specs_tables = bs.findChildren('table')
  if not specs_tables:
    error = '{} has no specs_tables| device_link: {}'.format(device_name, device_link)
    collect_debug.append(error)

  else:
    for table in specs_tables:
      table_rows = table.findChildren('tr', recursive=False)
      if table.findChild('th') == None:
        continue
      table_header = table.findChild('th').get_text(strip=True)
      ttl_collector = {}
      for row_num, row in enumerate(table_rows):
        ttl_tag = row.findChild('td', class_='ttl')
        nfo_tag = row.findChild('td', class_='nfo')

        if (not ttl_tag) or (not nfo_tag):
          error = '{} has ttl-tag OR nfo-tag| device_link:{}'.format(device_name, device_link)
          collect_debug(error)

          ttl_tag_text = np.NaN
          nfo_tag_text = np.NaN

        else:
          if not ttl_tag:
            error = '{} has no ttl-tag| device_link:{}'.format(device_name, device_link)
            collect_debug(error)
          else:
            ttl_tag_text = ttl_tag.get_text(strip=True)
            if ttl_tag_text == '\xa0' or ttl_tag_text == '':
              ttl_tag_text = np.NaN

          if not nfo_tag:
            error = 'No nfo-tag: {}: {}: {}'.format(n, link, row)
            collect_debug(error_mess)
          else:
            nfo_tag_text = nfo_tag.get_text(strip=True)
            if nfo_tag_text == '\xa0' or nfo_tag_text == '':
              nfo_tag_text = np.NaN

        ttl_collector.setdefault(ttl_tag_text, nfo_tag_text)

      specs_collector.setdefault(table_header, ttl_collector)
      
      
# checking
bs = get_bs(device_link)
get_device_specs(bs, specs_collector, device_name, device_link)
for key, value in specs_collector.items():
  print('{} : {}'.format(key, value))


Network : {'Technology': 'GSM / HSPA / LTE', '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)', '3G bands': 'HSDPA 850 / 900 / 2100', '4G bands': '1, 3, 5, 7, 8, 20, 28, 38, 40, 41', 'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'}
Launch : {'Announced': '2022, August 24', 'Status': 'Available. Released 2022, October 10'}
Body : {'Dimensions': '164.4 x 76.3 x 9.1 mm (6.47 x 3.00 x 0.36 in)', 'Weight': '192 g (6.77 oz)', 'Build': 'Glass front, plastic back, plastic frame', 'SIM': 'Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)'}
Display : {'Type': 'PLS LCD', 'Size': '6.5 inches, 102.0 cm2(~81.3% screen-to-body ratio)', 'Resolution': '720 x 1600 pixels, 20:9 ratio (~270 ppi density)'}
Platform : {'OS': 'Android 12, One UI Core 4.1', 'CPU': 'Octa-core (4x2.3 GHz & 4x1.8 GHz)'}
Memory : {'Card slot': 'microSDXC (dedicated slot)', 'Internal': '32GB 3GB RAM, 32GB 4GB RAM, 64GB 4GB RAM, 64GB 6GB RAM, 128GB 4GB RAM, 128GB 8GB RAM', nan: 'eMMC 5.1

In [12]:
def get_device_banner(bs, specs_collector, device_name, device_link):
  banner = bs.findChild('ul', class_='specs-spotlight-features')

  if not banner:
    error = '{} has no banner| device_link:{}'.format(device_name, device_link)
    collect_debug(error)

  else:
    banner_items = banner.findChildren('li')
    banner_specs_collector = {}
    for list_item in banner_items:

      banner_specs = list_item.findChildren(['span', 'div'], {'data-spec':re.compile('.*')})

      for banner_spec in banner_specs:
        banner_spec_name = banner_spec['data-spec']
        if banner_spec_name:

          banner_spec_value = banner_spec.get_text(strip=True)
          if banner_spec_value:
            banner_specs_collector[banner_spec_name] = banner_spec_value

      if 'help-popularity' in list_item['class']:
        
        device_popularity = list_item.findChild('strong')
        if not device_popularity:
          error = '{} has no device_popularity| device_link:{}'.format(device_name, device_link)
          collect_debug(error)
        else:
          device_popularity = device_popularity.get_text()
        
          if device_popularity == '\xa0' or device_popularity == '' :
            device_popularity = None        
        
        device_hits = list_item.findChild('span')
        if not device_hits:
          error = '{} has no device_hits| device_link:{}'.format(device_name, device_link)
          collect_debug(error)
        else:
          device_hits = device_hits.get_text()
          if device_hits == '\xa0'or device_hits == '':
            device_hits = None

        
        if device_popularity:
          banner_specs_collector['device_popularity'] = device_popularity
        if device_hits:
          banner_specs_collector['device_hits'] = device_hits
    specs_collector['Banner'] = banner_specs_collector

# checking
get_device_banner(bs, specs_collector, device_name, device_link)
for key, value in specs_collector.items():
  print('{} : {}'.format(key, value))
  

Network : {'Technology': 'GSM / HSPA / LTE', '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)', '3G bands': 'HSDPA 850 / 900 / 2100', '4G bands': '1, 3, 5, 7, 8, 20, 28, 38, 40, 41', 'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'}
Launch : {'Announced': '2022, August 24', 'Status': 'Available. Released 2022, October 10'}
Body : {'Dimensions': '164.4 x 76.3 x 9.1 mm (6.47 x 3.00 x 0.36 in)', 'Weight': '192 g (6.77 oz)', 'Build': 'Glass front, plastic back, plastic frame', 'SIM': 'Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)'}
Display : {'Type': 'PLS LCD', 'Size': '6.5 inches, 102.0 cm2(~81.3% screen-to-body ratio)', 'Resolution': '720 x 1600 pixels, 20:9 ratio (~270 ppi density)'}
Platform : {'OS': 'Android 12, One UI Core 4.1', 'CPU': 'Octa-core (4x2.3 GHz & 4x1.8 GHz)'}
Memory : {'Card slot': 'microSDXC (dedicated slot)', 'Internal': '32GB 3GB RAM, 32GB 4GB RAM, 64GB 4GB RAM, 64GB 6GB RAM, 128GB 4GB RAM, 128GB 8GB RAM', nan: 'eMMC 5.1

In [13]:
def get_device_opinions(bs,specs_collector, device_name, device_link):
  opinions = bs.findChild('div', id='opinions-total')
  if not opinions:
    error = '{} has no Total user opinions| device_link:{}'.format(devie_name, device_link)
    collect_debug(error)
  else:
    num_opinions = opinions.b.get_text(strip=True)
    specs_collector.setdefault('Opinions', num_opinions)
  

# checking
get_device_opinions(bs, specs_collector, device_name, device_link)
get_device_banner(bs, specs_collector, device_name, device_link)
for key, value in specs_collector.items():
  print('{} : {}'.format(key, value))

Network : {'Technology': 'GSM / HSPA / LTE', '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)', '3G bands': 'HSDPA 850 / 900 / 2100', '4G bands': '1, 3, 5, 7, 8, 20, 28, 38, 40, 41', 'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'}
Launch : {'Announced': '2022, August 24', 'Status': 'Available. Released 2022, October 10'}
Body : {'Dimensions': '164.4 x 76.3 x 9.1 mm (6.47 x 3.00 x 0.36 in)', 'Weight': '192 g (6.77 oz)', 'Build': 'Glass front, plastic back, plastic frame', 'SIM': 'Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)'}
Display : {'Type': 'PLS LCD', 'Size': '6.5 inches, 102.0 cm2(~81.3% screen-to-body ratio)', 'Resolution': '720 x 1600 pixels, 20:9 ratio (~270 ppi density)'}
Platform : {'OS': 'Android 12, One UI Core 4.1', 'CPU': 'Octa-core (4x2.3 GHz & 4x1.8 GHz)'}
Memory : {'Card slot': 'microSDXC (dedicated slot)', 'Internal': '32GB 3GB RAM, 32GB 4GB RAM, 64GB 4GB RAM, 64GB 6GB RAM, 128GB 4GB RAM, 128GB 8GB RAM', nan: 'eMMC 5.1

In [14]:
def get_device_data(device_link):
  
  specs_collector = {}
  
  bs = get_bs(device_link)
  device_name = bs.findChild('h1', class_='specs-phone-name-title').get_text()
  
  specs = get_device_specs(bs,specs_collector, device_name, device_link)
  
  banner = get_device_banner(bs, specs_collector, device_name, device_link)
  
  opinions = get_device_opinions(bs,specs_collector, device_name, device_link)
  
  return specs_collector

    
# checking
device_link = 'https://www.gsmarena.com/samsung_galaxy_s10-9536.php'
get_device_data(device_link)

{'Network': {'Technology': 'GSM / CDMA / HSPA / EVDO / LTE',
  '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)',
  nan: 'CDMA 800 / 1900 - USA',
  '3G bands': 'HSDPA 850 / 900 / 1700(AWS) / 1900 / 2100 - Global, USA',
  '4G bands': '1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 25, 26, 28, 32, 38, 39, 40, 41, 66 - Global',
  'Speed': 'HSPA 42.2/5.76 Mbps, LTE-A (7CA) Cat20 2000/150 Mbps'},
 'Launch': {'Announced': '2019, February 20',
  'Status': 'Available. Released 2019, March 08'},
 'Body': {'Dimensions': '149.9 x 70.4 x 7.8 mm (5.90 x 2.77 x 0.31 in)',
  'Weight': '157 g (5.54 oz)',
  'Build': 'Glass front (Gorilla Glass 6), glass back (Gorilla Glass 5), aluminum frame',
  'SIM': 'Single SIM (Nano-SIM) or Hybrid Dual SIM (Nano-SIM, dual stand-by)',
  nan: 'IP68 dust/water resistant (up to 1.5m for 30 mins)'},
 'Display': {'Type': 'Dynamic AMOLED, HDR10+',
  'Size': '6.1 inches, 93.2 cm2(~88.3% screen-to-body ratio)',
  'Resolution': '1440 x 3040 pixels

merging all

In [15]:

def get_maker_id(name_of_maker, maker_list):
  for maker in maker_list:
    name = maker[1]
    maker_id = maker[0]
    if name_of_maker.lower() == name.lower():
      return maker_id

  raise NameError('GSMArena has no maker \'{}\''.format(maker_name))
  
  
def switch(maker_id, name_of_maker, maker_list):
  if name_of_maker is None:
    return True
  else:
    given_maker_id = get_maker_id(name_of_maker, maker_list)
    if given_maker_id == maker_id:
      return True
    else:
      return False

In [16]:
def GSMCrawler(seed_url, name_of_maker=None):
  start_time = time.time()
# STEP 1: get the links to all the makers in GSMArena
  print('Starting GSMArena Crawler...\n')
  maker_list = get_maker_links(seed_url)
#   maker_list = get_maker_links(seed_url)
  print('Successfully retrived maker_list!\n')
  
  # tell us if we the crawl is being done for a single maker or all makers
  if name_of_maker is None:
    print('Crawling for devices by ALL makers...\n')
  else:
    print('Crawling for devices by {}...\n'.format(name_of_maker))
  
# STEP 2: iterate trough each maker and get the device links and device info from all the nav pages
  devices_collector = {}
  for maker_id, maker in enumerate(maker_list):
    if switch(maker_id, name_of_maker, maker_list):
      maker_link = maker[-1]
      maker_name = maker[1]

# STEP 3: the first thing we want to do on the makers page is to get a list of all nav links
      nav_pages_links = get_nav_page_links(maker)
      # for each nav page in a maker's nav_pages, get the device info for all devices by that maker
      print('Getting nav_page_links for {}...\n'.format(maker_name))
      devices_collector[maker_name] = []
      for nav_page_num, nav_page_link in nav_pages_links.items():
        get_device_links(nav_page_link, devices_collector, maker_name)
      print('Successfully collected all device info for {}!\n'.format(maker_name))
      
  # notify us of how many devices were collected in total 
  total_num_devices = 0
  for maker, devices_info in devices_collector.items():
    total_num_devices += devices_info.__len__()
  print('Successfully collected info for all devices! {} devices were collected\n'.format(total_num_devices))

# STEP 4: go through each each device_link in the devices_collector and pass it onto get_device_data
  print('Collecting spec sheets for all devices. This could take a while. Sit back and relax...\n')
# WARNING: This loop will scrape the spec sheet of every device in GSM Arena. it is good practice to put this under a try block; in case some thing we want to collect some debug info
  try:
    for maker, devices_info in devices_collector.items():
      print('Getting spec sheets for {} devices by {}\...n'.format(devices_info.__len__(), maker))
      for device_num, device in enumerate(devices_info):
        device_link = device[-1]

        device_specs =  get_device_data(device_link)
        device.append(device_specs)
# WARNING END
      print('Successfully scraped info for all devices by {}\n!'.format(maker))
  
  except Exception as e :
      error = 'Device crawl exception: {}| device_name: {}| device_links:{}\n'.format(e, device_link)
      collect_debug(error)

  else:
      end_time = time.time()
      print('GSMCrawler has completed excecuting! All credits for this data goes to the GSMArena team\n')
      print('Time time required to excecute for {}: {} seconds'.format(end_time - start_time))
      print('Time time per : {} seconds'.format(end_time - start_time))
      print('='*50)
  finally:

    return devices_collector

In [17]:
seed_path = 'makers.php3'
seed_url = domain_name + seed_path 

devices_collector = GSMCrawler(seed_url,'Samsung')

print(devices_collector.keys())
print(devices_collector['Samsung'])

Starting GSMArena Crawler...

Successfully retrived maker_list!

Crawling for devices by Samsung...

Getting nav_page_links for Samsung...

Successfully collected all device info for Samsung!

Successfully collected info for all devices! 1341 devices were collected


Getting spec sheets for 1341 devices by Samsung\...n
dict_keys(['Samsung'])
[['Galaxy A04e', 'Samsung Galaxy A04e Android smartphone. Announced Oct 2022. Features 6.5″  display, 5000 mAh battery, 128 GB storage, 4 GB RAM.', 'https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-a04e.jpg', 'https://www.gsmarena.com/samsung_galaxy_a04e-11945.php', {'Network': {'Technology': 'GSM / HSPA / LTE', '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)', '3G bands': 'HSDPA 850 / 900 / 2100', '4G bands': '1, 3, 5, 7, 8, 20, 28, 38, 40, 41', 'Speed': 'HSPA, LTE'}, 'Launch': {'Announced': '2022, October 21', 'Status': 'Available. Released 2022, November 07'}, 'Body': {'Dimensions': '164.2 x 75.9 x 9.1 mm (6.46 x

In [18]:
import json

def make_devices_json(devices_collector, save_json=False):
  
  json_dict = {}
  for maker, devices_info in devices_collector.items():
    maker_dict = {}
    for device_id, device in enumerate(devices_info):
      device_dict = {}
      if len(device) < 5:
        while len(device) != 5:
          device.append("NOT FOUND")
      print(len(device))
      device_name, device_info, device_img_link, device_link, device_specs = device

  
      device_dict['device_name'] = device_name
      device_dict['device_info'] = device_info
      device_dict['device_img_link'] = device_img_link
      device_dict['device_link'] = device_link
      device_dict['device_specs'] = device_specs


  
      maker_dict[device_id] = device_dict
  
    json_dict[maker] = maker_dict
    
  
  if save_json:
    cwd = os.getcwd()
    save_file_name = cwd + '/devices_data.txt'
    with open(save_file_name, 'w', encoding='utf-8') as file:
      json.dump(json_dict, file, ensure_ascii=False)
    
    print('Successfully saved device data as a JSON file at {}'.format(save_file_name))
    
  return json.dumps(json_dict, ensure_ascii=False)
  
# checking
devices_json = make_devices_json(devices_collector, save_json=True)
devices_json

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


'{"Samsung": {"0": {"device_name": "Galaxy A04e", "device_info": "Samsung Galaxy A04e Android smartphone. Announced Oct 2022. Features 6.5″  display, 5000 mAh battery, 128 GB storage, 4 GB RAM.", "device_img_link": "https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-a04e.jpg", "device_link": "https://www.gsmarena.com/samsung_galaxy_a04e-11945.php", "device_specs": {"Network": {"Technology": "GSM / HSPA / LTE", "2G bands": "GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)", "3G bands": "HSDPA 850 / 900 / 2100", "4G bands": "1, 3, 5, 7, 8, 20, 28, 38, 40, 41", "Speed": "HSPA, LTE"}, "Launch": {"Announced": "2022, October 21", "Status": "Available. Released 2022, November 07"}, "Body": {"Dimensions": "164.2 x 75.9 x 9.1 mm (6.46 x 2.99 x 0.36 in)", "Weight": "188 g (6.63 oz)", "Build": "Glass front, plastic back, plastic frame", "SIM": "Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)"}, "Display": {"Type": "PLS LCD", "Size": "6.5 inches, 102.0 cm2(~81.8% scree

In [19]:

!ls

devices_data.txt  sample_data


In [20]:
files.download('devices_data.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>